<a href="https://colab.research.google.com/github/yhc-666/hate-speech-detection/blob/master/randomforest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.metrics import make_scorer
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Jul 18 12:20:20 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
df_dataset = pd.read_csv('/content/drive/MyDrive/50.007kaggle/train.csv')
df_test = pd.read_csv('/content/drive/MyDrive/50.007kaggle/test.csv')
df_dataset_features = pd.read_csv("/content/drive/MyDrive/50.007kaggle/train_tfidf_features.csv")
df_test_features = pd.read_csv("/content/drive/MyDrive/50.007kaggle/test_tfidf_features.csv")
samplesubmit = pd.read_csv("/content/drive/MyDrive/50.007kaggle/sample_submission.csv")

In [ ]:
print(samplesubmit.shape)

(4296, 2)


In [ ]:
labels = df_dataset_features.iloc[:,[1]]
features = df_dataset_features.iloc[:,2:]
#y_test = df_test.iloc[:,[1]]
#X_test = df_test_features.iloc[:,1:]

#分训练集测试集
X_train, X_validation, y_train, y_validation = train_test_split(features, labels, test_size=0.05, random_state = 1000)

In [ ]:
print(X_train_lda)

[[-0.4312386 ]
 [ 1.5114172 ]
 [ 1.33581578]
 ...
 [-0.75423561]
 [-0.00360996]
 [ 0.56171887]]


In [ ]:
print(y_validation)
count = 0
for i in y_train['label']:
    if i == 1:
        count += 1
print(len(y_train) - count)
print(count/(len(y_train) - count))
print(count)
print(X_train)
print(X_validation)

       label
10162      0
5367       1
10077      0
11103      0
4837       1
...      ...
5291       1
3394       1
2858       1
14259      1
11420      0

[860 rows x 1 columns]
10111
0.6144792799920878
6213
         0    1    2    3    4    5    6    7    8    9  ...  4990  4991  \
14728  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   
197    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   
6844   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   
7608   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   
8985   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   
...    ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   ...   ...   
9274   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   
16840  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   
11863  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   
17093  0.0  0.0  0.0  0.0  0.0

In [ ]:
def my_custom_loss_func(y_true, y_pred):
    pos_f1 = metrics.f1_score(y_validation, y_pred_validation, pos_label=1 )
    neg_f1 = metrics.f1_score(y_validation, y_pred_validation, pos_label=0 )
    macro_f1 = (pos_f1 + neg_f1)/2  
    return macro_f1

macro_f1 = make_scorer(my_custom_loss_func, greater_is_better=True)

In [ ]:
model = RandomForestClassifier(n_estimators=100, 
                               #max_depth = 1000, 
                               min_samples_split = 5,
                               #max_leaf_nodes = 800,##
                               class_weight = {0:1, 1:2},
                               random_state = 4)

In [ ]:
model.fit(features,labels)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestClassifier(class_weight={0: 1, 1: 2}, min_samples_split=5,
                       random_state=4)

In [ ]:
print(model.decision_path)

<bound method BaseForest.decision_path of RandomForestClassifier(class_weight={0: 1, 1: 2}, min_samples_split=5,
                       random_state=4)>


In [ ]:
y_validation, y_pred_validation = y_validation, model.predict(X_validation)

threshold = 0.5
for i in range(len(y_pred_validation)):
    y_pred_validation[i] = 1 if y_pred_validation[i] > threshold else 0
pos_f1 = metrics.f1_score(y_validation, y_pred_validation, pos_label=1 )
neg_f1 = metrics.f1_score(y_validation, y_pred_validation, pos_label=0 )
macro_f1 = (pos_f1 + neg_f1)/2
print(pos_f1)
print(neg_f1)
print("validation_macro_f1 : %.4g" % macro_f1)  
print("validation_acc: %.4g"% metrics.accuracy_score(y_validation, y_pred_validation))

0.9966777408637874
0.998211091234347
validation_macro_f1 : 0.9974
validation_acc: 0.9977


In [ ]:
y_train, y_pred_train = y_train, model.predict(X_train)

threshold = 0.5
for i in range(len(y_pred_train)):
    y_pred_train[i] = 1 if y_pred_train[i] > threshold else 0
print(y_pred_train)
pos_f1 = metrics.f1_score(y_train, y_pred_train, pos_label=1 )
neg_f1 = metrics.f1_score(y_train, y_pred_train, pos_label=0 )
macro_f1 = (pos_f1 + neg_f1)/2
print(pos_f1)
print(neg_f1)
print("train_macro_f1 : %.4g" % macro_f1)  
print("train_acc: %.4g"% metrics.accuracy_score(y_train, y_pred_train))

[1 1 0 ... 0 0 0]
0.9931056597723265
0.9957370873401408
train_macro_f1 : 0.9944
train_acc: 0.9947


In [ ]:
X_test = df_test_features.iloc[:,1:]

In [ ]:
print(X_test)

        0    1    2    3    4    5    6    7    8    9  ...  4990  4991  4992  \
0     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   
1     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   
2     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   
3     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   
4     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   
...   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   ...   ...   ...   
4291  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   
4292  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   
4293  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   
4294  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   
4295  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   

      4993  4994  4995  499

In [ ]:
y_test = model.predict(X_test)

In [ ]:
print(y_test)

[1 0 1 ... 1 0 1]


In [ ]:
x = df_test
x = x.set_index("id")
x = x.drop("post", axis = 1)
x["label"] = y_test
print(x)
x.to_csv("/content/drive/MyDrive/50.007kaggle/submit.csv")

       label
id          
17185      1
17186      0
17187      1
17188      0
17189      0
...      ...
21476      1
21477      1
21478      1
21479      0
21480      1

[4296 rows x 1 columns]
